In [1]:
import sys, os
module_path = os.path.abspath(os.path.join('..'))
module_path = os.path.join(module_path, 'scripts')
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string

from utils.config import config
from utils.logger.logger import loggerCreator

## ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
#from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger


%load_ext autoreload

%autoreload 2

C:\Users\vipsharm\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vipsharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits \
            + string.ascii_lowercase) for _ in range(N))
    return str_



# Set Required Field for Parsing and Partition Modules

### Please ensure that your converted_html folder has html files in their specific language folders

Example: If your language code is en, please ensure that all html files reside in the converted_html/en folder. If folder is not present, a folder not found exception will be thrown

### English

In [ ]:
ePILanguage = 'en'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
procedureType = 'CAP'

### German

In [ ]:
ePILanguage = 'de'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
procedureType = 'CAP'

### Spanish

In [ ]:
ePILanguage = 'es'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
procedureType = 'CAP'

### Bulgarian

In [5]:
ePILanguage = 'bg'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
procedureType = 'CAP'

### Greek

In [6]:
ePILanguage = 'el'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
procedureType = 'CAP'

# Html Parsing Stage

In [ ]:
class FolderNotFoundError(Exception):
    pass

## Generate input folder path
module_path = os.path.abspath(os.path.join('..'))
module_path = os.path.join(module_path, 'data')
module_path = os.path.join(module_path, 'converted_to_html')
module_path = os.path.join(module_path, ePILanguage)

## Generate output folder path
output_json_path = module_path.replace('converted_to_html','outputJSON')

"""
    Check if input folder exists, else throw exception
"""
if(os.path.exists(module_path)):
    filenames = glob.glob(os.path.join(module_path, '*.html'))
    filenames.extend(glob.glob(os.path.join(module_path, '*.htm')))
    
    ## Create language specific folder in outputJSON folder if it doesn't exist
    if(not os.path.exists(output_json_path)):
        os.mkdir(output_json_path)
    logger = loggerCreator('Parser_'+ getRandomString(1))
    
    styleRulesObj = StyleRulesDictionary(loggerCreator('Style Dictionary_'+ getRandomString(1)),
                                     language = ePILanguage,
                                     fileName = fileNameQrd,
                                     procedureType = procedureType)

    parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict, 
                            styleRulesObj.styleFeatureKeyList, 
                            styleRulesObj.qrd_section_headings)

    for input_filename in filenames:
      #if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
        output_filename = input_filename.replace('converted_to_html','outputJSON')
        output_filename = output_filename.replace('.html','.json')
        output_filename = output_filename.replace('.htm','.json')
        print(input_filename, output_filename)
        parserObj.createPIJsonFromHTML(input_filepath = input_filename,
                                       output_filepath = output_filename,
                                       img_base64_dict= parserObj.convertImgToBase64(input_filename),
                                      )
else:
    raise FolderNotFoundError(module_path + " not found")

# Partition Stage

In [ ]:
styleRulesObj = StyleRulesDictionary(loggerCreator('Style Dictionary_'+ getRandomString(1)), 
                                     language = ePILanguage,
                                     fileName = fileNameQrd,
                                     procedureType = procedureType)

path_json = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'outputJSON', ePILanguage)

partitionlogger = loggerCreator('Partition_'+ getRandomString(1))
partitioner = DocTypePartitioner(partitionlogger)
partitioner.partitionHtmls(styleRulesObj.qrd_section_headings, path_json)

# Matching Stage

In [ ]:
procedureType = 'CAP'
languageCode = 'es'
stopWordlanguage = 'spanish'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'
fileNameDoc = 'emea-combined-h-2494-es_ PROSPECTO.json'

In [ ]:
### SmPC
documentNumber = 3
docFilter = "PROSPECTO.json"
stopWordFilterLen = 100
start=0
end=1
isPackageLeaflet=True
topHeadingsConsidered = 5
bottomHeadingsConsidered = 10

In [ ]:
 os.environ['APPLICATIONINSIGHTS_CONNECTION_STRING'] = "InstrumentationKey=769acdf5-503c-43e6-9736-77925ec553f0"

In [ ]:
os.path.join(os.path.abspath(os.path.join('..')),'utils','matchLog.txt')

In [ ]:
logger = MatchLogger("Matching Logger",fileNameDoc, procedureType, languageCode, documentNumber, fileNameLog = os.path.join(os.path.abspath(os.path.join('..')), 'code','utils','matchLog.txt'))
    

In [ ]:
matchDocObj = MatchDocument(
                 logger,
                 procedureType,
                 languageCode,
                 documentNumber,
                 fileNameDoc,
                 fileNameQrd,
                 fileNameMatchRuleBook,
                 fileNameDocumentTypeNames,
                 topHeadingsConsidered,
                 bottomHeadingsConsidered,
                 stopWordFilterLen,
                 stopWordlanguage,
                 isPackageLeaflet,
                 'Kalydeco')
df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()

# Content Extraction Stage

In [ ]:
extractContentlogger = loggerCreator('ExtractContentBetween_'+ getRandomString(1))
extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll)
dfExtractedHierRR = extractorObj.extractContentBetweenHeadings('Abilify-h-471-e_ PACKAGE LEAFLET.json')

# XML Generation Stage

In [ ]:
xmlLogger = loggerCreator('XmlGeneration_'+ getRandomString(1))
fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger)
fhirXmlGeneratorObj.generateXml(dfExtractedHierRR,  'Abilify-h-471-e_ PACKAGE LEAFLET.xml')

In [ ]:
convertCollectionToDataFrame(coll)

In [ ]:
documentAnnotationObj = DocumentAnnotation('Kalydeco II-86-PI-clean_SmPC.json','c270d6ccaf9e47e9b20b322e2383c4ba','https://spor-uat.azure-api.net/pms/api/v2/','df','coll')

In [ ]:
d = documentAnnotationObj.processRegulatedAuthorizationForDoc(['EU/2/10/106/006','EU/2/09/098/001'])
d

In [ ]:
d['Author Value']

In [ ]:
d['Medicinal Product Definitions']

In [ ]:
documentAnnotationObj.processRegulatedAuthorizationForDoc(['EU/3/00/001','EU/1/97/039/003'])

# End-To-End Flow 

In [5]:
def parseDocuments(procedureType,
                   languageCode,
                   documentNumber,
                   docFilter,
                   fileNameQrd,
                   fileNameMatchRuleBook,
                   fileNameDocumentTypeNames,
                   stopWordFilterLen=6,
                   start=0,
                   end=10,
                   isPackageLeaflet=False,
                   medName=None):

    '''
    This function was created to run the match function on multiple documents in automated fashion.
    
    '''
    if documentNumber == 0:
        topHeadingsConsidered=4
        bottomHeadingsConsidered=6
    elif documentNumber == 1:
        topHeadingsConsidered=3
        bottomHeadingsConsidered=5
    elif documentNumber == 2:
        topHeadingsConsidered=5
        bottomHeadingsConsidered=15
    else:
        topHeadingsConsidered=5
        bottomHeadingsConsidered=10
                
    path_json = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'partitionedJSONs',f'{languageCode}')
    print(path_json)
    _,_,fileNames = next(os.walk(path_json))
    print(fileNames)
    filteredNames = [ fileName for fileName in fileNames if docFilter in fileName]
    filteredNames = filteredNames[start:end]
    print(filteredNames)
    fileNameLog = os.path.join(os.path.abspath(os.path.join('..')), 'code','utils','matchLog.txt')
    for fileNameDoc in filteredNames:
        
        
        print(f"Starting Heading Extraction For File :- {fileNameDoc}")
        logger = MatchLogger(f"match logger {fileNameDoc}", fileNameDoc, procedureType, languageCode, documentNumber, fileNameLog )
        logger.logFlowCheckpoint("Starting Heading Extraction")
        
        matchDocObj = MatchDocument(
                 logger,
                 procedureType,
                 languageCode,
                 documentNumber,
                 fileNameDoc,
                 fileNameQrd,
                 fileNameMatchRuleBook,
                 fileNameDocumentTypeNames,
                 topHeadingsConsidered,
                 bottomHeadingsConsidered,
                 stopWordFilterLen,
                 stopWordlanguage,
                 isPackageLeaflet,
                 medName)
        df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()
        
        print(f"Completed Heading Extraction For File")
        logger.logFlowCheckpoint("Completed Heading Extraction For File")

        print(f"Starting Document Annotation For File :- {fileNameDoc}")        
        logger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNameDoc,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except:
            pms_oms_annotation_data = None
            print("Error Found")
            
        print(f"Completed Document Annotation")        
        logger.logFlowCheckpoint("Completed Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNameDoc}")        
        logger.logFlowCheckpoint("Starting Extracting Content Between Heading")

        #extractContentlogger = loggerCreator('ExtractContentBetween_'+ getRandomString(1))
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{documentNumber}', fileNameDoc, procedureType, languageCode, documentNumber, fileNameLog)

        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll, languageCode)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNameDoc)
        
        print(f"Completed Extracting Content Between Heading")        
        logger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        
        #xmlLogger = loggerCreator('XmlGeneration_'+ getRandomString(1))
        xmlLogger =  MatchLogger(f'XmlGeneration_{documentNumber}', fileNameDoc, procedureType, languageCode, documentNumber, fileNameLog)

        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger)
        fileNameXml = fileNameDoc.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR,  fileNameXml)
        
        #fhirServiceObj = FhirService(generatedXml)
        #fhirServiceObj.submitFhirXml()
        print(f"Created XML File For :- {fileNameDoc}")        

        return df,coll,dfExtractedHierRR


In [6]:
def parseDocuments(fileNameHtml,
                   domain,
                   procedureType,
                   languageCode,
                   documentNumber,
                   docFilter,
                   fileNameQrd,
                   fileNameMatchRuleBook,
                   fileNameDocumentTypeNames,
                   stopWordFilterLen=6,
                   start=0,
                   end=10,
                   isPackageLeaflet=False,
                   medName=None,
                   fsMountName = '/mounted',
                   localEnv= True):
    
    '''
    This function was created to run the match function on multiple documents in automated fashion.
    
    '''
    
    stylesFilePath = os.path.join(os.path.abspath(os.path.join('..')),'data','outputJSON',domain, procedureType, languageCode, fileNameHtml)
    stylesFilePath =  stylesFilePath.replace('.html','.txt')
    stylesFilePath =  stylesFilePath.replace('.txtl','.txt')
    stylesFilePath =  stylesFilePath.replace('.htm','.txt')
    
    fileNameLog = os.path.join(os.path.abspath(os.path.join('..')),'data','FinalLog.txt')

    if documentNumber == 0:
        topHeadingsConsidered=4
        bottomHeadingsConsidered=6
    elif documentNumber == 1:
        topHeadingsConsidered=3
        bottomHeadingsConsidered=5
    elif documentNumber == 2:
        topHeadingsConsidered=5
        bottomHeadingsConsidered=15
    else:
        topHeadingsConsidered=5
        bottomHeadingsConsidered=10
                
    path_json = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'partitionedJSONs',f'{domain}',f'{procedureType}',f'{languageCode}')
    print(path_json)
    _,_,fileNames = next(os.walk(path_json))
    print(fileNames)
    filteredNames = [ fileName for fileName in fileNames if docFilter in fileName]
    filteredNames = filteredNames[start:end]
    print(filteredNames)
    fileNameLog = os.path.join(os.path.abspath(os.path.join('..')), 'code','utils','matchLog.txt')
    for index, fileNamePartitioned in enumerate(filteredNames):
        
        
                                      
        flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", fileNamePartitioned, domain, procedureType, languageCode, "HTML", fileNameLog)
        if documentNumber == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll = extractAndValidateHeadings(domain,
                                    procedureType,
                                    languageCode,
                                    documentNumber,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName,
                                    fsMountName=fsMountName,
                                    localEnv=localEnv)
        
        
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        
        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except:
            pms_oms_annotation_data = None
            print("Error Found")
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{documentNumber}', fileNamePartitioned, domain, procedureType, languageCode, documentNumber, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll, domain, procedureType, languageCode, fsMountName, localEnv)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{documentNumber}', fileNamePartitioned, domain, procedureType, languageCode, documentNumber, fileNameLog)
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, pms_oms_annotation_data, stylesFilePath, medName, fsMountName, localEnv)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{documentNumber}', fileNamePartitioned, domain, procedureType, languageCode, documentNumber, fileNameLog)

        fhirServiceObj = FhirService(fhirServiceLogger, generatedXml, fsMountName, localEnv)
        fhirServiceObj.submitFhirXml()
        print(f"Created XML File For :- {fileNamePartitioned}")           

        return df,coll,dfExtractedHierRR


In [7]:
fileNameQrd = 'qrd_canonical_model.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'

fsMountName = '/mounted'

In [8]:
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames


class FolderNotFoundError(Exception):
    pass


def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv):

    if localEnv is True:
        pathSep = "\\"
        module_path = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    else:
        pathSep = "/"
        module_path = os.path.join(f'{fsMountName}', 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    
    # Generate output folder path
    output_json_path = module_path.replace('converted_to_html', 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, fileNameHtml))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', fileNameHtml,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)

        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = input_filename.replace('converted_to_html', 'outputJSON')
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
            
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)
    if localEnv is True:
        path_json = os.path.join(os.path.abspath(os.path.join(
                '..')), 'data', 'outputJSON', domain, procedureType, languageCode, fileNameJson)
    else:
        path_json = os.path.join(f'{fsMountName}', 'data', 'outputJSON', domain,  procedureType, languageCode, fileNameJson)
        
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger, localEnv)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(domain,
                               procedureType,
                               languageCode,
                               documentNumber,
                               fileNameDoc,
                               fileNameQrd,
                               fileNameMatchRuleBook,
                               fileNameDocumentTypeNames,
                               fileNameLog,
                               stopWordFilterLen=6,
                               isPackageLeaflet=False,
                               medName=None,
                               fsMountName='/mounted',
                               localEnv=False):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber,
        fsMountName=fsMountName,
        localEnv=localEnv).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName,
        fsMountName,
        localEnv)
    df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll


def parseDocument(htmlDocPath, fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, fsMountName = '/mounted', localEnv= False, medName = None):
    
    if localEnv is True:
        pathSep = "\\"
        fileNameLog = os.path.join(os.path.abspath(os.path.join('..')),'data','FinalLog.txt')
    else:
        pathSep = "/"
        fileNameLog = os.path.join(f'{fsMountName}','data','FinalLog.txt')
    pathComponents = htmlDocPath.split(pathSep)
    print(pathComponents)
    fileNameHtml = pathComponents[-1]
    languageCode =  pathComponents[-2]
    procedureType = pathComponents[-3]
    domain = pathComponents[-4]

    #procedureType = "CAP"
    print(fileNameHtml,languageCode,procedureType)
    
    
    
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    fileNameJson, stylesFilePath = convertHtmlToJson( domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll = extractAndValidateHeadings(domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName,
                                    fsMountName=fsMountName,
                                    localEnv=localEnv)
        
        
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        
        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except:
            pms_oms_annotation_data = None
            print("Error Found")
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll, domain, procedureType, languageCode, fsMountName, localEnv)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, pms_oms_annotation_data, stylesFilePath, medName, fsMountName, localEnv)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)

        fhirServiceObj = FhirService(fhirServiceLogger, generatedXml, fsMountName, localEnv)
        fhirServiceObj.submitFhirXml()
        print(f"Created XML File For :- {fileNamePartitioned}")        

        #return df,coll,dfExtractedHierRR
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")


In [14]:
def convertToInt(x):
    try:
        return str(int(x))
    except:
        return x


def convertCollectionToDataFrame(collection):

    dfExtractedHier = pd.DataFrame(collection)
    dfExtractedHier['parent_id'] = dfExtractedHier['parent_id'].apply(
        lambda x: convertToInt(x))
    dfExtractedHier['id'] = dfExtractedHier['id'].apply(
        lambda x: convertToInt(x))

    return dfExtractedHier

In [ ]:
####
#Comman Parameters
procedureType = 'CAP'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'

## Spanish

In [ ]:
languageCode = 'es'

### Kalydeco



### SmPC

In [ ]:
documentNumber = 0
docFilter = "SmPC.json"
stopWordFilterLen = 6
stopWordlanguage = 'spanish'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

In [ ]:
convertCollectionToDataFrame(b)

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

In [ ]:
from QrdExtractor.qrdExtractor import QrdCanonical

q = QrdCanonical(fileNameQrd, procedureType, languageCode, 'SmPC').ProcessQrdDataframe()

In [ ]:
r = q.loc[1]

In [ ]:
"�".encode()

In [ ]:
(dfQrd[dfQrd['id'] == r['parent_id']]['heading_id'])[0]
    

In [ ]:
q.to_dict('records')

In [ ]:
documentAnnotationObj = DocumentAnnotation('emea-combined-h-2494-es_SmPC.json','c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',a,b)



In [ ]:
documentAnnotationObj.processRegulatedAuthorizationForDoc(['',''])

### Annex II

In [ ]:
documentNumber = 1
docFilter = "ANEXO II.json"
stopWordFilterLen = 6
stopWordlanguage = 'spanish'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

### Labeling

In [ ]:
documentNumber = 2
docFilter = "ANEXO III.json"
stopWordFilterLen = 6
stopWordlanguage = 'spanish'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

### Package Leaflet

In [ ]:

documentNumber = 3
docFilter = "PROSPECTO.json"
stopWordFilterLen = 100
stopWordlanguage = 'spanish'
start=0
end=1
isPackageLeaflet=True

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end,
               isPackageLeaflet,
               'Kalydeco'
              )

## German

In [ ]:
languageCode = 'de'

### Kalydeco



### SmPC

In [ ]:
os.environ['APPLICATIONINSIGHTS_CONNECTION_STRING'] = 'InstrumentationKey=769acdf5-503c-43e6-9736-77925ec553f0'

In [ ]:
documentNumber = 0
docFilter = "SmPC.json"
stopWordFilterLen = 6
stopWordlanguage = 'german'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

In [ ]:
a.head(1)

In [ ]:
a

In [ ]:
from match.rulebook.matchRulebook import MatchRuleBook

rules= MatchRuleBook(
            fileNameRuleBook= fileNameMatchRuleBook,
            procedureType= procedureType,
            languageCode= languageCode,
            documentNumber= 0).ruleDict



### Annex II

In [ ]:
documentNumber = 1
docFilter = "ANHANG II.json"
stopWordFilterLen = 6
stopWordlanguage = 'german'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

### Labeling

In [ ]:
documentNumber = 2
docFilter = "ANHANG III.json"
stopWordFilterLen = 6
stopWordlanguage = 'german'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

In [ ]:
a

### Package Leaflet

In [ ]:
documentNumber = 3
docFilter = "PACKUNGSBEILAGE.json"
stopWordFilterLen = 100
stopWordlanguage = 'german'
start=0
end=1
isPackageLeaflet = True

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end,
               isPackageLeaflet,
               "Kalydeco"
              )

## Bulgarian

In [14]:
languageCode = 'bg'

### Kalydeco



### SmPC

In [15]:
os.environ['APPLICATIONINSIGHTS_CONNECTION_STRING'] = 'InstrumentationKey=769acdf5-503c-43e6-9736-77925ec553f0'

In [16]:
domain = 'H'
documentNumber = 0
docFilter = "SmPC.json"
stopWordFilterLen = 6
stopWordlanguage = 'english'
start=0
end=1

In [17]:
a,b,c = parseDocuments('emea-combined-h-2494-bg.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

2021-05-11 05:59:48,779 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Starting Heading Extraction | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json
2021-05-11 05:59:48,979 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Started Extracting Heading | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json


F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg
['emea-combined-h-2494-bg_SmPC.json', 'emea-combined-h-2494-bg_Б. ЛИСТОВКА.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json']
['emea-combined-h-2494-bg_SmPC.json']
Starting Heading Extraction For File :- emea-combined-h-2494-bg_SmPC.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg\emea-combined-h-2494-bg_SmPC.json
--------------------------------------------
documentTypeNames.json bg H CAP 0 /mounted True
SmPC


2021-05-11 05:59:49,136 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Match Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- 'КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА' | Qrd txt :- 'КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА' | Matched :- 'True'
2021-05-11 05:59:49,141 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Passed As This The First Heading | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '1' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 05:59:49,182 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Match Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- '1.       ИМЕ НА ЛЕКАРСТВЕНИЯ ПРОДУКТ' | Qrd txt :- '1. ИМЕ НА ЛЕКАРСТВЕНИЯ ПРОДУКТ' | Matched :- 'True'
2021-05-11 05:59:49,190 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Flow Is Broken | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '3' | currParentHeadId :- '1.0'


OriginalCheck



2021-05-11 05:59:51,456 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Match Failed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- 'Всяка филмирана таблетка съдържа 83,6 mg лактоза монохидрат.' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False'
2021-05-11 05:59:52,505 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Match Failed In Lowercase | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- 'Помощно вещество с известно действие' | Qrd txt :- 'Помощно(и) вещество(а) с известно действие' | Matched :- 'False'


...



Heading Not Found 
 ['qТози лекарствен продукт подлежи на допълнително наблюдение. Това ще позволи бързото установяване на нова информация относно безопасността. От медицинските специалисти се изисква да съобщават всяка подозирана нежелана реакция. За начина на съобщаване на нежелани реакции вижте точка 4.8', 'Общо описание', 'Качествен и количествен състав', 'Помощно(и) вещество(а) с известно действие', 'Предпазни мерки, които трябва да бъдат взети преди работа със или приложение на лекарствения продукт', 'Проследимост', 'Връзка(и) фармакокинетика-фармакодинамика', 'Оценка на риска за околната среда (ERA)', 'Употреба при педиатричната популация', 'ДОЗИМЕТРИЯ', 'ИНСТРУКЦИИ ЗА ПРИГОТВЯНЕ НА РАДИОФАРМАЦЕВТИЦИ ']


dict_keys([])
Completed Heading Extraction For File
Starting Document Annotation For File :- emea-combined-h-2494-bg_SmPC.json
Error Found
Completed Document Annotation
Starting Extracting Content Between Heading For File :- emea-combined-h-2494-bg_SmPC.json
File being proces

2021-05-11 06:02:01,131 : Flow Logger HTML_j : Completed Extracting Content Between Heading | H | CAP |  bg | HTML | emea-combined-h-2494-bg_SmPC.json
2021-05-11 06:02:01,152 : XmlGeneration_0 : PMS/OMS Annotation Information Not Retrieved | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json
2021-05-11 06:02:01,155 : XmlGeneration_0 : Initiating XML Generation | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json


Completed Extracting Content Between Heading
Already Exists


2021-05-11 06:02:01,914 : XmlGeneration_0 : Writing to File:emea-combined-h-2494-bg_SmPC.xml | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json
2021-05-11 06:02:06,861 : XML Submission Logger_0 : Initiating Submission To FHIR Server | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json
2021-05-11 06:02:06,865 : XML Submission Logger_0 : Response{"resourceType":"Bundle","id":"8b8eecda-90cb-426e-9316-7843534c6672","meta":{"versionId":"1","lastUpdated":"2021-05-11T00:32:05.118+00:00"},"type":"collection","entry":[{"fullUrl":"urn:uuid:f05f046d-ad74-4533-8456-63aa097ac0a0","resource":{"resourceType":"Bundle","id":"091d05f4-f7e7-49c2-afc4-96462659c729","identifier":{"system":"http://ema.europa.eu/fhir/identifier/documentid","value":"${instance.bundle[n].Identifier}"},"type":"document","timestamp":"2021-05-11T00:32:01+00:00","entry":[{"fullUr | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json
2021-05-11 06:02:06,878 : XML Submission Logger_0 : POST sucessful: XML added with id: 8b

POST sucessful: XML added with id 8b8eecda-90cb-426e-9316-7843534c6672
Created XML File For :- emea-combined-h-2494-bg_SmPC.json


In [ ]:
convertCollectionToDataFrame(b)

### Annex II

In [11]:
documentNumber = 1
docFilter = "ПРИЛОЖЕНИЕ II.json"
stopWordFilterLen = 6
stopWordlanguage = 'english'
start=0
end=1
domain = 'H'
procedureType = 'CAP'

In [12]:
a,b,c = parseDocuments('emea-combined-h-2494-bg.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end)



2021-05-11 05:46:33,168 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Starting Heading Extraction | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
2021-05-11 05:46:33,251 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Started Extracting Heading | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
2021-05-11 05:46:33,259 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Passed | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | Doc txt :- 'ПРИЛОЖЕНИЕ II' | Qrd txt :- 'ПРИЛОЖЕНИЕ II' | Matched :- 'True'
2021-05-11 05:46:33,266 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Validation Passed As This The First Heading | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | currHeadId :- '1001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 05:46:33,291 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Passed | H | CAP |

F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg
['emea-combined-h-2494-bg_SmPC.json', 'emea-combined-h-2494-bg_Б. ЛИСТОВКА.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json']
['emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json']
Starting Heading Extraction For File :- emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg\emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
--------------------------------------------
documentTypeNames.json bg H CAP 1 /mounted True
AnnexII

OriginalCheck



2021-05-11 05:46:33,411 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Failed In Lowercase | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | Doc txt :- 'Б.         УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА ДОСТАВКА И УПОТРЕБА ' | Qrd txt :- 'Име и адрес на производителя(ите), отговорен(ни) за освобождаване на партидите' | Matched :- 'False'
2021-05-11 05:46:33,413 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Passed | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | Doc txt :- 'Б.         УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА ДОСТАВКА И УПОТРЕБА ' | Qrd txt :- 'Б. УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА ДОСТАВКА И УПОТРЕБА' | Matched :- 'True'
2021-05-11 05:46:33,422 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Validation Flow Is Broken | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | currHeadId :- '1005' | currParentHeadId :- '1001.0' | prevParentHeadId :- '1002'
2021-05-11 05:46:33,427 : Heading Extra


OriginalCheck

----------------------------------
RemovedByStyle
----------------------------------
----------------------------------
RemovedByStyle
----------------------------------

OriginalCheck


OriginalCheck



2021-05-11 05:46:33,688 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Failed In Lowercase | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | Doc txt :- 'В.         ДРУГИ УСЛОВИЯ И ИЗИСКВАНИЯ НА РАЗРЕШЕНИЕТО ЗА УПОТРЕБА' | Qrd txt :- 'A. <ПРОИЗВОДИТЕЛ(И) НА БИОЛОГИЧНО АКТИВНОТО(ИТЕ) ВЕЩЕСТВО(А) И>ПРОИЗВОДИТЕЛ(И), ОТГОВОРЕН(НИ) ЗА ОСВОБОЖДАВАНЕ НА ПАРТИДИ ' | Matched :- 'False'
2021-05-11 05:46:33,781 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Failed | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | Doc txt :- 'В.         ДРУГИ УСЛОВИЯ И ИЗИСКВАНИЯ НА РАЗРЕШЕНИЕТО ЗА УПОТРЕБА' | Qrd txt :- 'Б. УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА ДОСТАВКА И УПОТРЕБА' | Matched :- 'False'


...



OriginalCheck


OriginalCheck



2021-05-11 05:46:40,418 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Failed | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | Doc txt :- '·                Задължение за провеждане на мерки след разрешаване за употреба' | Qrd txt :- 'Допълнителни мерки за свеждане на риска до минимум' | Matched :- 'False'
2021-05-11 05:46:40,420 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Match Passed | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | Doc txt :- '·                Задължение за провеждане на мерки след разрешаване за употреба' | Qrd txt :- 'Задължение за провеждане на мерки след разрешаване за употреба' | Matched :- 'True'
2021-05-11 05:46:40,429 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json : Validation Flow Is Broken | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json | currHeadId :- '1012' | currParentHeadId :- '1009.0' | prevParentHeadId :- '1010'
2021-05-11 05:46:40,434 :


OriginalCheck



Heading Not Found 
 ['Име и адрес на производителя(ите) на биологично активното(ите) вещество(а)', 'УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА ДОСТАВКА И УПОТРЕБА', 'Официално освобождаване на партиди', 'ДРУГИ УСЛОВИЯ И ИЗИСКВАНИЯ НА РАЗРЕШЕНИЕТО ЗА УПОТРЕБА', 'УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА БЕЗОПАСНА И ЕФЕКТИВНА УПОТРЕБА НА ЛЕКАРСТВЕНИЯ ПРОДУКТ', 'Допълнителни мерки за свеждане на риска до минимум', 'КОНКРЕТНО ЗАДЪЛЖЕНИЕ ЗА ИЗПЪЛНЕНИЕ НА МЕРКИ СЛЕД <РАЗРЕШАВАНЕ ЗА УПОТРЕБА ПОД УСЛОВИЕ> <РАЗРЕШАВАНЕ ЗА УПОТРЕБА ПРИ ИЗВЪНРЕДНИ ОБСТОЯТЕЛСТВА>']


dict_keys(['Б. УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА ДОСТАВКА И УПОТРЕБА', 'Г. УСЛОВИЯ ИЛИ ОГРАНИЧЕНИЯ ЗА БЕЗОПАСНА И ЕФЕКТИВНА УПОТРЕБА НА ЛЕКАРСТВЕНИЯ ПРОДУКТ', 'В. ДРУГИ УСЛОВИЯ И ИЗИСКВАНИЯ НА РАЗРЕШЕНИЕТО ЗА УПОТРЕБА', 'Име и адрес на производителя(ите), отговорен(ни) за освобождаване на партидите'])
Completed Heading Extraction For File
Starting Document Annotation For File :- emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
Error Found
Completed Document An

2021-05-11 05:46:40,821 : XmlGeneration_1 : Writing to File:emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.xml | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
2021-05-11 05:46:42,652 : XML Submission Logger_1 : Initiating Submission To FHIR Server | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
2021-05-11 05:46:42,655 : XML Submission Logger_1 : Response{"resourceType":"Bundle","id":"a0a42b62-6580-4e05-a9f3-fec0314e251c","meta":{"versionId":"1","lastUpdated":"2021-05-11T00:16:42.236+00:00"},"type":"collection","entry":[{"fullUrl":"urn:uuid:93963110-5255-406b-8b61-7c577255c4d7","resource":{"resourceType":"Bundle","id":"48776f97-30f9-4564-a7fc-4df85fe8c5f3","identifier":{"system":"http://ema.europa.eu/fhir/identifier/documentid","value":"${instance.bundle[n].Identifier}"},"type":"document","timestamp":"2021-05-11T00:16:40+00:00","entry":[{"fullUr | H | CAP |  bg | 1 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json
2021-05-11 05:46:42,660 : XML Submission Logger_1 : P

POST sucessful: XML added with id a0a42b62-6580-4e05-a9f3-fec0314e251c
Created XML File For :- emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json


In [ ]:
a

In [15]:
convertCollectionToDataFrame(b)

,index,id,Procedure type,Display code,Name,parent_id,htmlText,htmlIndex,htmlId,SubSectionIndex,doc_parent_id
0,60,1001,CAP,NaN,ПРИЛОЖЕНИЕ II,NaN,ПРИЛОЖЕНИЕ II,25,dcc7e187-914c-42e9-bfba-f53e3f6db06c,0,
1,61,1002,CAP,A.,"<ПРОИЗВОДИТЕЛ(И) НА БИОЛОГИЧНО АКТИВНОТО(ИТЕ) ВЕЩЕСТВО(А) И>ПРОИЗВОДИТЕЛ(И), ОТГОВОРЕН(НИ) ЗА ОСВОБОЖДАВАНЕ НА ПАРТИДИ",1001,"A. ПРОИЗВОДИТЕЛ(И), ОТГОВОРЕН(НИ) ЗА ОСВОБОЖДАВАНЕ НА ПАРТИДИ",27,1a3c1329-9123-465d-a41f-0b86766d84d0,0,1001
2,61,1002,CAP,A.,"<ПРОИЗВОДИТЕЛ(И) НА БИОЛОГИЧНО АКТИВНОТО(ИТЕ) ВЕЩЕСТВО(А) И>ПРОИЗВОДИТЕЛ(И), ОТГОВОРЕН(НИ) ЗА ОСВОБОЖДАВАНЕ НА ПАРТИДИ",1001,"A. ПРОИЗВОДИТЕЛ(И), ОТГОВОРЕН(НИ) ЗА ОСВОБОЖДАВАНЕ НА ПАРТИДИ",42,92afb852-db9d-4c76-8447-468e55a10e48,0,1001
3,63,1004,CAP,NaN,"Име и адрес на производителя(ите), отговорен(ни) за освобождаване на партидите",1002,"Име и адрес на производителя(ите), отговорен(ни) за освобождаване на партидите",44,95990ece-be8a-414b-b6d5-df0930253505,0,1002
4,67,1008,CAP,NaN,Периодични актуализирани доклади за безопасност (ПАДБ),1007,· Периодични актуализирани доклади за безопасност (ПАДБ),68,9c96bf35-d2ea-41f1-84b9-2f3f2b04091a,0,1004
5,69,1010,CAP,NaN,План за управление на риска (ПУР),1009,· План за управление на риска (ПУР),75,448249f3-f890-444c-a5cb-d33bdacb45ee,0,1008
6,71,1012,CAP,NaN,Задължение за провеждане на мерки след разрешаване за употреба,1009,· Задължение за провеждане на мерки след разрешаване за употреба,88,fa7a555c-7d28-47a6-ac17-f229c748454a,0,1010


In [ ]:
'\u043f\u043e \u0438\u0441\u043a\u0430\u043d\u0435      \u043d\u0430      \u0415\u0432\u0440\u043e\u043f\u0435\u0439\u0441\u043a\u0430\u0442\u0430      \u0430\u0433\u0435\u043d\u0446\u0438\u044f \u043f\u043e      \u043b\u0435\u043a\u0430\u0440\u0441\u0442\u0432\u0430\u0442\u0430;'.encode().decode()

In [ ]:
"\u041f\u043e\u0434\u0440\u043e\u0431\u043d\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044f \u0437\u0430 \u0442\u043e\u0432\u0430 \u043b\u0435\u043a\u0430\u0440\u0441\u0442\u0432o \u0435 \u043f\u0440\u0435\u0434\u043e\u0441\u0442\u0430\u0432\u0435\u043d\u0430 \u043d\u0430 \u0443\u0435\u0431\u0441\u0430\u0439\u0442\u0430 \u043d\u0430 \u0415\u0432\u0440\u043e\u043f\u0435\u0439\u0441\u043a\u0430\u0442\u0430 \u0430\u0433\u0435\u043d\u0446\u0438\u044f \u043f\u043e \u043b\u0435\u043a\u0430\u0440\u0441\u0442\u0432\u0430\u0442\u0430 http:\/\/www.ema.europa.eu. \u041f\u043e\u0441\u043e\u0447\u0435\u043d\u0438 \u0441\u0430 \u0441\u044a\u0449\u043e \u043b\u0438\u043d\u043a\u043e\u0432\u0435 \u043a\u044a\u043c \u0434\u0440\u0443\u0433\u0438 \u0443\u0435\u0431\u0441\u0430\u0439\u0442\u043e\u0432\u0435, \u043a\u044a\u0434\u0435\u0442\u043e \u043c\u043e\u0436\u0435 \u0434\u0430 \u0441\u0435 \u043d\u0430\u043c\u0435\u0440\u0438 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044f \u0437\u0430 \u0440\u0435\u0434\u043a\u0438 \u0437\u0430\u0431\u043e\u043b\u044f\u0432\u0430\u043d\u0438\u044f \u0438 \u043b\u0435\u0447\u0435\u043d\u0438\u044f.".encode().decode()

### Labeling

In [32]:
documentNumber = 2
docFilter = "ПРИЛОЖЕНИЕ III.json"
stopWordFilterLen = 6
stopWordlanguage = 'english'
start=0
end=1

In [33]:
a,b,c = parseDocuments('emea-combined-h-2494-bg.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

2021-05-11 05:51:43,749 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Starting Heading Extraction | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json
2021-05-11 05:51:43,897 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Started Extracting Heading | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json


F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg
['emea-combined-h-2494-bg_SmPC.json', 'emea-combined-h-2494-bg_Б. ЛИСТОВКА.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json']
['emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json']
Starting Heading Extraction For File :- emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg\emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json
--------------------------------------------
documentTypeNames.json bg H CAP 2 /mounted True
Labelling


2021-05-11 05:51:44,086 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Passed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'данни върху опаковката и листовка ' | Qrd txt :- 'данни върху опаковката' | Matched :- 'True'
2021-05-11 05:51:44,089 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Passed In Lowercase | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ДАННИ ВЪРХУ ОПАКОВКАТА И ЛИСТОВКА ' | Qrd txt :- 'ДАННИ ВЪРХУ ОПАКОВКАТА' | Matched :- 'True'
2021-05-11 05:51:44,099 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Validation Passed As This The First Heading | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | currHeadId :- '2001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 05:51:44,112 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Passed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛ


OriginalCheck



2021-05-11 05:51:44,388 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ВЪНШНА КАРТОНЕНА ОПАКОВКА ЗА БЛИСТЕРИ – ОПАКОВКА ОТ 56 ТАБЛЕТКИ' | Qrd txt :- '13. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' | Matched :- 'False'
2021-05-11 05:51:44,501 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ВЪНШНА КАРТОНЕНА ОПАКОВКА ЗА БЛИСТЕРИ – ОПАКОВКА ОТ 56 ТАБЛЕТКИ' | Qrd txt :- '4. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' | Matched :- 'False'
2021-05-11 05:51:44,556 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ВЪНШНА КАРТОНЕНА ОПАКОВКА ЗА БЛИСТЕРИ – ОПАКОВКА ОТ 56 ТАБЛЕТКИ' | Qrd txt :- '4. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' 

...

2021-05-11 05:51:57,692 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ДАННИ, КОИТО ТРЯБВА ДА СЪДЪРЖА ВТОРИЧНАТА ОПАКОВКА' | Qrd txt :- '13. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' | Matched :- 'False'
2021-05-11 05:51:57,819 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ДАННИ, КОИТО ТРЯБВА ДА СЪДЪРЖА ВТОРИЧНАТА ОПАКОВКА' | Qrd txt :- '4. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' | Matched :- 'False'
2021-05-11 05:51:57,871 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ДАННИ, КОИТО ТРЯБВА ДА СЪДЪРЖА ВТОРИЧНАТА ОПАКОВКА' | Qrd txt :- '4. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' | Matched :- 'False'
2021-05-11 05:51:5

oooooooooooooooooooooooooooooooooooooooo END OF Sub Section oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo


2021-05-11 05:51:58,198 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ВЪНШНА КАРТОНЕНА ОПАКОВКА ЗА БЛИСТЕР-КАРТА – ОПАКОВКА ОТ 28 ТАБЛЕТКИ' | Qrd txt :- '13. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' | Matched :- 'False'
2021-05-11 05:51:58,311 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ВЪНШНА КАРТОНЕНА ОПАКОВКА ЗА БЛИСТЕР-КАРТА – ОПАКОВКА ОТ 28 ТАБЛЕТКИ' | Qrd txt :- '4. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И НА ПРОДУКТА>' | Matched :- 'False'
2021-05-11 05:51:58,394 : Heading Extraction emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json : Match Failed | H | CAP |  bg | 2 | emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json | Doc txt :- 'ВЪНШНА КАРТОНЕНА ОПАКОВКА ЗА БЛИСТЕР-КАРТА – ОПАКОВКА ОТ 28 ТАБЛЕТКИ' | Qrd txt :- '4. ПАРТИДЕН НОМЕР<, КОДОВЕ НА ДАРЕНИЕТО И


OriginalCheck




KeyboardInterrupt: 

In [ ]:
a[a['Text'].str.find('НАЧИН НА ПРИЛОЖЕНИЕ И ПЪТ(ИЩА) НА ВЪВЕЖДАНЕ')!=-1]

In [ ]:
convertCollectionToDataFrame(b)

### Package Leaflet

In [12]:
documentNumber = 3
docFilter = "ЛИСТОВКА.json"
stopWordFilterLen = 100
stopWordlanguage = 'english'
start=0
end=1
isPackageLeaflet = True
domain= 'H'

In [13]:
a,b,c = parseDocuments('emea-combined-h-2494-bg.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end,
               isPackageLeaflet=True,
               medName="Kalydeco",
               
                       
              )

2021-05-11 05:53:56,676 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Starting Heading Extraction | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
2021-05-11 05:53:56,793 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Started Extracting Heading | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
2021-05-11 05:53:56,806 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Passed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Б. ЛИСТОВКА' | Qrd txt :- ' ЛИСТОВКА' | Matched :- 'True'
2021-05-11 05:53:56,810 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Validation Passed As This The First Heading | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | currHeadId :- '3001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 05:53:56,816 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed In Lowercase | H | CAP |  bg | 3 | 

F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg
['emea-combined-h-2494-bg_SmPC.json', 'emea-combined-h-2494-bg_Б. ЛИСТОВКА.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ II.json', 'emea-combined-h-2494-bg_ПРИЛОЖЕНИЕ III.json']
['emea-combined-h-2494-bg_Б. ЛИСТОВКА.json']
Starting Heading Extraction For File :- emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg\emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
--------------------------------------------
documentTypeNames.json bg H CAP 3 /mounted True
Package leaflet

OriginalCheck



2021-05-11 05:53:56,879 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Листовка: информация за пациента' | Qrd txt :- 'Бременност <и> <,> кърмене <и фертилитет>' | Matched :- 'False'
2021-05-11 05:53:56,990 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Листовка: информация за пациента' | Qrd txt :- 'Дата на последно преразглеждане на листовката <{ММ /ГГГГ}> <{месец ГГГГ}>.' | Matched :- 'False'
2021-05-11 05:53:57,061 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Kalydeco 75 mg филмирани таблетки' | Qrd txt :- 'Бременност <и> <,> кърмене <и фертилитет>' | Matched :- 'False'
2021-05-11 05:53:57,146 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Fa


OriginalCheck



2021-05-11 05:53:58,250 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- '·                Запазете тази листовка. Може да се наложи да я прочетете отново.' | Qrd txt :- 'Дата на последно преразглеждане на листовката <{ММ /ГГГГ}> <{месец ГГГГ}>.' | Matched :- 'False'
2021-05-11 05:53:58,400 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- '·                Ако имате някакви допълнителни въпроси, попитайте Вашия лекар или фармацевт.' | Qrd txt :- 'Бременност <и> <,> кърмене <и фертилитет>' | Matched :- 'False'
2021-05-11 05:53:58,612 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- '·                Ако имате някакви допълнителни въпроси, попитайте Вашия лекар или фармацевт.' | Qr

...

2021-05-11 05:56:05,257 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Други източници на информация' | Qrd txt :- 'Дата на последно преразглеждане на листовката <{ММ /ГГГГ}> <{месец ГГГГ}>.' | Matched :- 'False'
2021-05-11 05:56:05,259 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Passed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Други източници на информация' | Qrd txt :- 'Други източници на информация' | Matched :- 'True'
2021-05-11 05:56:05,270 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Validation Passed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | currHeadId :- '3029' | currParentHeadId :- '3023.0' | prevParentHeadId :- '3028'


2021-05-11 05:56:05,647 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Подробна информация за това лекарствo е предоставена на уебсайта на Европейската агенция по лекарствата http://www.ema.europa.eu. Посочени са също линкове към други уебсайтове, където може да се намери информация за редки заболявания и лечения.' | Qrd txt :- 'Бременност <и> <,> кърмене <и фертилитет>' | Matched :- 'False'
2021-05-11 05:56:06,095 : Heading Extraction emea-combined-h-2494-bg_Б. ЛИСТОВКА.json : Match Failed | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json | Doc txt :- 'Подробна информация за това лекарствo е предоставена на уебсайта на Европейската агенция по лекарствата http://www.ema.europa.eu. Посочени са също линкове към други уебсайтове, където може да се намери информация за редки заболявания и лечения.' | Qrd txt :- 'Дата на последно преразглеждане на листовката <{ММ /ГГГГ}> <



Heading Not Found 
 ['q Този лекарствен продукт подлежи на допълнително наблюдение. Това ще позволи бързото установяване на нова информация относно безопасността. Можете да дадете своя принос като съобщите всяка нежелана реакция, която сте получили. За начина на съобщаване на нежелани реакции вижте края на точка 4. ', 'Какво представлява Х и за какво се използва', 'Не <приемайте> <използвайте> X', 'Други лекарства и Х', 'Притежател на разрешението за употреба и производител', 'За допълнителна информация относно това лекарствo, моля, свържете се с локалния представител на притежателя на разрешението за употреба:', 'Посочената по-долу информация е предназначена само за медицински специалисти:']


dict_keys([])
Completed Heading Extraction For File
Starting Document Annotation For File :- emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
Error Found
Completed Document Annotation
Starting Extracting Content Between Heading For File :- emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
File being processed:

2021-05-11 05:56:06,509 : XmlGeneration_3 : Writing to File:emea-combined-h-2494-bg_Б. ЛИСТОВКА.xml | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
2021-05-11 05:56:09,454 : XML Submission Logger_3 : Initiating Submission To FHIR Server | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
2021-05-11 05:56:09,457 : XML Submission Logger_3 : Response{"resourceType":"Bundle","id":"43a0179c-bdc4-4d90-9b8c-6ef29a4607b9","meta":{"versionId":"1","lastUpdated":"2021-05-11T00:26:08.622+00:00"},"type":"collection","entry":[{"fullUrl":"urn:uuid:4bf8cab8-8459-49b9-9b18-c5c69cc9d3b6","resource":{"resourceType":"Bundle","id":"01911e2d-663a-44cf-bad6-d8cbecabd4be","identifier":{"system":"http://ema.europa.eu/fhir/identifier/documentid","value":"${instance.bundle[n].Identifier}"},"type":"document","timestamp":"2021-05-11T00:26:06+00:00","entry":[{"fullUr | H | CAP |  bg | 3 | emea-combined-h-2494-bg_Б. ЛИСТОВКА.json
2021-05-11 05:56:09,466 : XML Submission Logger_3 : POST suce

POST sucessful: XML added with id 43a0179c-bdc4-4d90-9b8c-6ef29a4607b9
Created XML File For :- emea-combined-h-2494-bg_Б. ЛИСТОВКА.json


In [18]:
convertCollectionToDataFrame(b)

NameError: name 'b' is not defined

## Greek

In [9]:
languageCode = 'el'
procedureType = "CAP"

### Kalydeco



### SmPC

In [10]:
documentNumber = 0
docFilter = "SmPC.json"
stopWordFilterLen = 6
stopWordlanguage = 'greek'
start=0
end=1
domain= 'H'


In [11]:
a,b,c = parseDocuments('emea-combined-h-2494-el.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end,
               isPackageLeaflet=True,
               medName="Kalydeco",
               
                       
              )

2021-05-11 06:22:59,130 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Starting Heading Extraction | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:22:59,335 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Started Extracting Heading | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json


F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el
['emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json', 'emea-combined-h-2494-el_SmPC.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json']
['emea-combined-h-2494-el_SmPC.json']
Starting Heading Extraction For File :- emea-combined-h-2494-el_SmPC.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el\emea-combined-h-2494-el_SmPC.json
--------------------------------------------
documentTypeNames.json el H CAP 0 /mounted True
SmPC


2021-05-11 06:22:59,490 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Match Passed | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | Doc txt :- 'ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ' | Qrd txt :- 'ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ' | Matched :- 'True'
2021-05-11 06:22:59,494 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Validation Passed As This The First Heading | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | currHeadId :- '16001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 06:22:59,522 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Match Passed | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | Doc txt :- '1.       ΟΝΟΜΑΣΙΑ ΤΟΥ ΦΑΡΜΑΚΕΥΤΙΚΟΥ ΠΡΟΪΟΝΤΟΣ' | Qrd txt :- '1. ΟΝΟΜΑΣΙΑ ΤΟΥ ΦΑΡΜΑΚΕΥΤΙΚΟΥ ΠΡΟΪΟΝΤΟΣ' | Matched :- 'True'
2021-05-11 06:22:59,530 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Validation Flow Is Broken | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | currH

2021-05-11 06:23:00,152 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Match Passed | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | Doc txt :- '2.       ΠΟΙΟΤΙΚΗ ΚΑΙ ΠΟΣΟΤΙΚΗ ΣΥΝΘΕΣΗ' | Qrd txt :- '2. ΠΟΙΟΤΙΚΗ ΚΑΙ ΠΟΣΟΤΙΚΗ ΣΥΝΘΕΣΗ' | Matched :- 'True'
2021-05-11 06:23:00,162 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Validation Passed | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | currHeadId :- '16004' | currParentHeadId :- '16001.0' | prevParentHeadId :- '16003'


2021-05-11 06:23:00,958 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Match Passed | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | Doc txt :- 'Έκδοχο με γνωστή δράση:' | Qrd txt :- 'Έκδοχο(α) με γνωστή δράση' | Matched :- 'True'
2021-05-11 06:23:00,972 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Validation Flow Is Broken | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | currHeadId :- '16007' | currParentHeadId :- '16006.0' | prevParentHeadId :- '16004'
2021-05-11 06:23:00,983 : Heading Extraction emea-combined-h-2494-el_SmPC.json : Validation Passed | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json | currHeadId :- '16007' | currParentHeadId :- '16006.0' | prevParentHeadId :- '16004'


...



Heading Not Found 
 ['qΤο φάρμακο αυτό τελεί υπό συμπληρωματική παρακολούθηση. Αυτό θα επιτρέψει το γρήγορο προσδιορισμό νέων πληροφοριών ασφάλειας. Ζητείται από τους επαγγελματίες υγείας να αναφέρουν οποιεσδήποτε πιθανολογούμενες ανεπιθύμητες ενέργειες. Βλ. παράγραφο 4.8 για τον τρόπο αναφοράς ανεπιθύμητων ενεργειών.', 'Γενική περιγραφή', 'Ποιοτική και ποσοτική σύνθεση', 'Προφυλάξεις που πρέπει να ληφθούν πριν από τον χειρισμό ή τη χορήγηση του φαρμακευτικού προϊόντος', 'Ιχνηλασιμότητα', 'Κύηση', 'Φαρμακοκινητικές/φαρμακοδυναμικές σχέσεις', 'Αξιολόγηση περιβαλλοντικού κινδύνου', 'Χρήση στον παιδιατρικό πληθυσμό', 'ΔΟΣΙΜΕΤΡΙΑ', 'ΟΔΗΓΙΕΣ ΠΑΡΑΣΚΕΥΗΣ ΡΑΔΙΟΦΑΡΜΑΚΩΝ']


dict_keys([])
Completed Heading Extraction For File
Starting Document Annotation For File :- emea-combined-h-2494-el_SmPC.json
 ['EU/1/12/782/001', 'EU/1/12/782/002', 'EU/1/12/782/005', 'EU/1/12/782/007', 'EU/1/12/782/003', 'EU/1/12/782/004', 'EU/1/12/782/006']
EU/1/12/782/001


2021-05-11 06:25:12,247 : Flow Logger HTML_n : Completed Document Annotation | H | CAP |  el | HTML | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:12,249 : Flow Logger HTML_n : Starting Extracting Content Between Heading | H | CAP |  el | HTML | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:12,254 : ExtractContentBetween_0 : Cleaning Match Results | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:12,260 : ExtractContentBetween_0 : Finished Cleaning Match Results | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:12,426 : Flow Logger HTML_n : Completed Extracting Content Between Heading | H | CAP |  el | HTML | emea-combined-h-2494-el_SmPC.json


Error Found
Completed Document Annotation
Starting Extracting Content Between Heading For File :- emea-combined-h-2494-el_SmPC.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el\emea-combined-h-2494-el_SmPC.json
--------------------------------------------
Completed Extracting Content Between Heading
Already Exists


2021-05-11 06:25:12,465 : XmlGeneration_0 : PMS/OMS Annotation Information Not Retrieved | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:12,467 : XmlGeneration_0 : Initiating XML Generation | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:13,231 : XmlGeneration_0 : Writing to File:emea-combined-h-2494-el_SmPC.xml | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:17,684 : XML Submission Logger_0 : Initiating Submission To FHIR Server | H | CAP |  el | 0 | emea-combined-h-2494-el_SmPC.json
2021-05-11 06:25:17,686 : XML Submission Logger_0 : Response{"resourceType":"Bundle","id":"04ce29f1-d083-4634-96f7-748919af4da0","meta":{"versionId":"1","lastUpdated":"2021-05-11T00:55:15.954+00:00"},"type":"collection","entry":[{"fullUrl":"urn:uuid:103fced7-02de-4718-ae3a-798d9b71c5e8","resource":{"resourceType":"Bundle","id":"db8db760-5a65-4f65-a822-da0e88edf6d4","identifier":{"system":"http://ema.europa.eu/fhir/identifier/docume

POST sucessful: XML added with id 04ce29f1-d083-4634-96f7-748919af4da0
Created XML File For :- emea-combined-h-2494-el_SmPC.json


In [ ]:
from match.rulebook.matchRulebook import MatchRuleBook

r = MatchRuleBook(fileNameMatchRuleBook, procedureType, languageCode, documentNumber)

from match.matchStrings.matchStrings import MatchStrings
m = MatchStrings(logger, documentNumber, r, 6, "greek")

In [ ]:
m.matchStrings('Κύηση', 'Κύηση',23, avoidLowerCaseMatch=False)

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

In [ ]:
'Έκδοχο(α) με γνωστή δράση'.encode().decode('utf-8')

### Annex II

In [12]:
documentNumber = 1
docFilter = "ΠΑΡΑΡΤΗΜΑ ΙΙ.json"
stopWordFilterLen = 6
stopWordlanguage = 'greek'
start=0
end=1


In [13]:
a,b,c = parseDocuments('emea-combined-h-2494-el.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end                     
              )

2021-05-11 06:25:18,102 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Starting Heading Extraction | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
2021-05-11 06:25:18,186 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Started Extracting Heading | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
2021-05-11 06:25:18,194 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Passed | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | Doc txt :- 'ΠΑΡΑΡΤΗΜΑ ΙΙ' | Qrd txt :- 'ΠΑΡΑΡΤΗΜΑ ΙΙ' | Matched :- 'True'
2021-05-11 06:25:18,199 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Validation Passed As This The First Heading | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | currHeadId :- '17001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 06:25:18,222 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Passed | H | CAP |  el | 1 |

F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el
['emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json', 'emea-combined-h-2494-el_SmPC.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json']
['emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json']
Starting Heading Extraction For File :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el\emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
--------------------------------------------
documentTypeNames.json el H CAP 1 /mounted True
ΠΑΡΑΡΤΗΜΑ ΙΙ



2021-05-11 06:25:18,311 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Passed | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | Doc txt :- 'Β.          ΟΡΟΙ Ή ΠΕΡΙΟΡΙΣΜΟΙ ΣΧΕΤΙΚΑ ΜΕ ΤΗ ΔΙΑΘΕΣΗ ΚΑΙ ΤΗ ΧΡΗΣΗ' | Qrd txt :- 'B. ΟΡΟΙ Ή ΠΕΡΙΟΡΙΣΜΟΙ ΣΧΕΤΙΚΑ ΜΕ ΤΗ ΔΙΑΘΕΣΗ ΚΑΙ ΤΗ ΧΡΗΣΗ ' | Matched :- 'True'
2021-05-11 06:25:18,323 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Validation Flow Is Broken | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | currHeadId :- '17005' | currParentHeadId :- '17001.0' | prevParentHeadId :- '17002'
2021-05-11 06:25:18,328 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Validation Passed | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | currHeadId :- '17005' | currParentHeadId :- '17001.0' | prevParentHeadId :- '17002'
2021-05-11 06:25:18,394 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Failed In Lowercase | H | CAP |  el | 1 | e



OriginalCheck


OriginalCheck



2021-05-11 06:25:18,610 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Passed | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | Doc txt :- 'δ.          ΟΡΟΙ Ή ΠΕΡΙΟΡΙΣΜΟΙ ΣΧΕΤΙΚΑ ΜΕ ΤΗΝ ΑΣΦΑΛΗ ΚΑΙ ΑΠΟΤΕΛΕΣΜΑΤΙΚΗ ΧΡΗΣΗ ΤΟΥ ΦΑΡΜΑΚΕΥΤΙΚΟΥ ΠΡΟΪΟΝΤΟΣ' | Qrd txt :- 'B. ΟΡΟΙ Ή ΠΕΡΙΟΡΙΣΜΟΙ ΣΧΕΤΙΚΑ ΜΕ ΤΗ ΔΙΑΘΕΣΗ ΚΑΙ ΤΗ ΧΡΗΣΗ ' | Matched :- 'True'
2021-05-11 06:25:18,621 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Validation Failed As Wrong Heading Found | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | currHeadId :- '17005' | currParentHeadId :- '17001.0' | prevParentHeadId :- '17007'
2021-05-11 06:25:18,673 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Failed In Lowercase | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | Doc txt :- 'δ.          ΟΡΟΙ Ή ΠΕΡΙΟΡΙΣΜΟΙ ΣΧΕΤΙΚΑ ΜΕ ΤΗΝ ΑΣΦΑΛΗ ΚΑΙ ΑΠΟΤΕΛΕΣΜΑΤΙΚΗ ΧΡΗΣΗ ΤΟΥ ΦΑΡΜΑΚΕΥΤΙΚΟΥ ΠΡΟΪΟΝΤΟΣ' | Qrd txt :- 'C. ΑΛΛΟΙ ΟΡΟΙ ΚΑΙ ΑΠΑΙΤΗΣΕ

...


OriginalCheck




2021-05-11 06:25:21,810 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Passed | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | Doc txt :- '·        Σχέδιο διαχείρισης κινδύνου (ΣΔΚ)' | Qrd txt :- 'Σχέδιο διαχείρισης κινδύνου (ΣΔΚ)' | Matched :- 'True'
2021-05-11 06:25:21,824 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Validation Passed | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | currHeadId :- '17010' | currParentHeadId :- '17009.0' | prevParentHeadId :- '17009'
2021-05-11 06:25:22,281 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Match Passed | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json | Doc txt :- '·        Υποχρέωση λήψης μετεγκριτικών μέτρων' | Qrd txt :- 'Υποχρέωση λήψης μετεγκριτικών μέτρων ' | Matched :- 'True'
2021-05-11 06:25:22,295 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json : Validation Flow Is Broken | H | CAP |  el | 1 | emea-combined


OriginalCheck



Heading Not Found 
 ['Όνομα και διεύθυνση του(των) παρασκευαστή(ών) της(των) βιολογικώς δραστικής(ών) ουσίας(ών)', 'Όνομα και διεύθυνση του(των) παρασκευαστή(ών) που είναι υπεύθυνος(οι) για την αποδέσμευση των παρτίδων', 'Επίσημη αποδέσμευση παρτίδων', 'Εκθέσεις περιοδικής παρακολούθησης της ασφάλειας (PSURs)', 'Επιπρόσθετα μέτρα ελαχιστοποίησης κινδύνου', 'ΕΙΔΙΚΗ ΥΠΟΧΡΕΩΣΗ ΟΛΟΚΛΗΡΩΣΗΣ ΜΕΤΕΓΚΡΙΤΙΚΩΝ ΜΕΤΡΩΝ ΓΙΑ <ΤΗΝ ΑΔΕΙΑ ΚΥΚΛΟΦΟΡΙΑΣ ΜΕ ΕΓΚΡΙΣΗ ΥΠΟ ΟΡΟΥΣ> <ΤΗΝ ΑΔΕΙΑ ΚΥΚΛΟΦΟΡΙΑΣ ΥΠΟ ΕΞΑΙΡΕΤΙΚΕΣ ΠΕΡΙΣΤΑΣΕΙΣ>']


dict_keys([])
Completed Heading Extraction For File
Starting Document Annotation For File :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
Error Found
Completed Document Annotation
Starting Extracting Content Between Heading For File :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el\emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
--------------------------------------------

2021-05-11 06:25:24,604 : XML Submission Logger_1 : Initiating Submission To FHIR Server | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
2021-05-11 06:25:24,606 : XML Submission Logger_1 : Response{"resourceType":"Bundle","id":"9b4a1dd9-f4ed-4230-b7a0-d1e2f24d386e","meta":{"versionId":"1","lastUpdated":"2021-05-11T00:55:24.053+00:00"},"type":"collection","entry":[{"fullUrl":"urn:uuid:b13e174b-9654-4594-9d78-bcd772c02864","resource":{"resourceType":"Bundle","id":"ef4f4eb9-37d5-490e-ae68-bcb8e9ec44a7","identifier":{"system":"http://ema.europa.eu/fhir/identifier/documentid","value":"${instance.bundle[n].Identifier}"},"type":"document","timestamp":"2021-05-11T00:55:22+00:00","entry":[{"fullUr | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json
2021-05-11 06:25:24,609 : XML Submission Logger_1 : POST sucessful: XML added with id: 9b4a1dd9-f4ed-4230-b7a0-d1e2f24d386e | H | CAP |  el | 1 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json


POST sucessful: XML added with id 9b4a1dd9-f4ed-4230-b7a0-d1e2f24d386e
Created XML File For :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json


### Labeling

In [14]:
documentNumber = 2
docFilter = "ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json"
stopWordFilterLen = 6
stopWordlanguage = 'english'
start=0
end=1

In [15]:
a,b,c = parseDocuments('emea-combined-h-2494-el.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end                     
              )

2021-05-11 06:25:25,068 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json : Starting Heading Extraction | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json


F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el
['emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json', 'emea-combined-h-2494-el_SmPC.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json']
['emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json']
Starting Heading Extraction For File :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el\emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
--------------------------------------------
documentTypeNames.json

2021-05-11 06:25:25,404 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json : Started Extracting Heading | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json


 el H CAP 2 /mounted True
ΕΠΙΣΗΜΑΝΣΗ


2021-05-11 06:25:26,078 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json : Match Passed | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json | Doc txt :- 'A. ΕΠΙΣΗΜΑΝΣΗ' | Qrd txt :- 'ΕΠΙΣΗΜΑΝΣΗ' | Matched :- 'True'
2021-05-11 06:25:26,088 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json : Validation Passed As This The First Heading | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json | currHeadId :- '18001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 06:25:26,110 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json : Match Passed | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json | Doc txt :- 'ΕΝΔΕΙΞΕΙΣ ΠΟΥ ΠΡΕΠΕΙ ΝΑ ΑΝΑΓΡΑΦΟΝΤΑΙ ΣΤΗΝ ΕΞΩΤΕΡΙΚΗ ΣΥΣΚΕΥΑΣΙΑ' | Qrd txt :- 'ΕΝΔΕΙΞΕΙΣ ΠΟΥ ΠΡΕΠΕΙ ΝΑ ΑΝΑΓΡΑΦΟΝΤΑΙ <ΣΤΗΝ ΕΞΩΤΕΡΙΚΗ ΣΥΣΚΕΥΑΣΙΑ> <ΚΑΙ> < ΣΤΗ ΣΤΟΙΧΕΙΩΔΗ ΣΥΣΚΕΥΑΣΙΑ>' | Matched :- 'True'
2021-05-11 06:25:26,124 : Heading Extraction emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json : Validation

...


All mandatory headings have been found !!!

dict_keys([])
Completed Heading Extraction For File
Starting Document Annotation For File :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
Error Found
Completed Document Annotation
Starting Extracting Content Between Heading For File :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el\emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
--------------------------------------------


2021-05-11 06:27:56,843 : Flow Logger HTML_d : Completed Extracting Content Between Heading | H | CAP |  el | HTML | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
2021-05-11 06:27:56,871 : XmlGeneration_2 : PMS/OMS Annotation Information Not Retrieved | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
2021-05-11 06:27:56,873 : XmlGeneration_2 : Initiating XML Generation | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json


Completed Extracting Content Between Heading
Already Exists


2021-05-11 06:27:57,228 : XmlGeneration_2 : Writing to File:emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.xml | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
2021-05-11 06:28:00,094 : XML Submission Logger_2 : Initiating Submission To FHIR Server | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
2021-05-11 06:28:00,097 : XML Submission Logger_2 : Response{"resourceType":"Bundle","id":"b5ea9360-8c19-426e-bee7-f52cc7f8f937","meta":{"versionId":"1","lastUpdated":"2021-05-11T00:57:59.311+00:00"},"type":"collection","entry":[{"fullUrl":"urn:uuid:f1729b74-8015-46ba-b82a-f7835fb63de8","resource":{"resourceType":"Bundle","id":"f5fd62c1-5fd0-45de-ad8b-dbfdbbf5be14","identifier":{"system":"http://ema.europa.eu/fhir/identifier/documentid","value":"${instance.bundle[n].Identifier}"},"type":"document","timestamp":"2021-05-11T00:57:56+00:00","entry":[{"fullUr | H | CAP |  el | 2 | emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json
2021-05-11 06:28:00,102 : XML Submission Logger_2 : P

POST sucessful: XML added with id b5ea9360-8c19-426e-bee7-f52cc7f8f937
Created XML File For :- emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json


In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Package Leaflet

In [16]:
documentNumber = 3
docFilter = "ΧΡΗΣΗΣ.json"
stopWordFilterLen = 100
stopWordlanguage = 'english'
start=0
end=1
isPackageLeaflet = True

In [ ]:
a,b,c = parseDocuments('emea-combined-h-2494-el.html',
               domain,
               procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end,
               True,
               "Kalydeco"
              )

2021-05-11 06:28:00,466 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Starting Heading Extraction | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-05-11 06:28:00,627 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Started Extracting Heading | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-05-11 06:28:00,646 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Passed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'B. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ' | Qrd txt :- 'ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ' | Matched :- 'True'


F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el
['emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json', 'emea-combined-h-2494-el_SmPC.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙ.json', 'emea-combined-h-2494-el_ΠΑΡΑΡΤΗΜΑ ΙΙΙ.json']
['emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json']
Starting Heading Extraction For File :- emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\el\emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
--------------------------------------------
documentTypeNames.json el H CAP 3 /mounted True
ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ


2021-05-11 06:28:00,654 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Validation Passed As This The First Heading | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '19001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
2021-05-11 06:28:00,792 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Φύλλο οδηγιών χρήσης: Πληροφορίες για τον ασθενή' | Qrd txt :- 'Κύηση<,> <και> θηλασμός <και γονιμότητα>' | Matched :- 'False'
2021-05-11 06:28:01,124 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Passed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Φύλλο οδηγιών χρήσης: Πληροφορίες για τον ασθενή' | Qrd txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις <{ΜΜ/ΕΕΕΕ}> <{μήνας ΕΕΕΕ}.>' | Matched :- 'True'
2021-

2021-05-11 06:28:10,110 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Passed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Τι περιέχει το παρόν φύλλο οδηγιών:' | Qrd txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις <{ΜΜ/ΕΕΕΕ}> <{μήνας ΕΕΕΕ}.>' | Matched :- 'True'
2021-05-11 06:28:10,123 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Validation Passed As Previous Heading Same As Current | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '19028' | currParentHeadId :- '19023.0' | prevParentHeadId :- '19028'
2021-05-11 06:28:10,257 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed In Lowercase | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- '1.    Τι είναι το Kalydeco και ποια είναι η χρήση του' | Qrd txt :- '1. Τι είναι το Χ και ποια είναι η χρήση του'


OriginalCheck



...

2021-05-11 06:31:44,600 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Vertex Pharmaceuticals (Ireland) Limited' | Qrd txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις <{ΜΜ/ΕΕΕΕ}> <{μήνας ΕΕΕΕ}.>' | Matched :- 'False'
2021-05-11 06:31:44,693 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- '28-32 Pembroke Street Upper' | Qrd txt :- 'Κύηση<,> <και> θηλασμός <και γονιμότητα>' | Matched :- 'False'
2021-05-11 06:31:44,818 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- '28-32 Pembroke Street Upper' | Qrd txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις <{ΜΜ/ΕΕΕΕ}> <{μήν


OriginalCheck



2021-05-11 06:31:45,807 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Almac Pharma Services (Ireland) Limited' | Qrd txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις <{ΜΜ/ΕΕΕΕ}> <{μήνας ΕΕΕΕ}.>' | Matched :- 'False'
2021-05-11 06:31:45,913 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Finnabair Industrial Estate' | Qrd txt :- 'Κύηση<,> <και> θηλασμός <και γονιμότητα>' | Matched :- 'False'
2021-05-11 06:31:46,012 : Heading Extraction emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json : Match Failed | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Finnabair Industrial Estate' | Qrd txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις <{ΜΜ/ΕΕΕΕ}> <{μήνα

2021-05-11 06:31:49,413 : ExtractContentBetween_3 : Cleaning Match Results | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-05-11 06:31:49,422 : ExtractContentBetween_3 : Finished Cleaning Match Results | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-05-11 06:31:49,506 : Flow Logger HTML_i : Completed Extracting Content Between Heading | H | CAP |  el | HTML | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-05-11 06:31:49,530 : XmlGeneration_3 : PMS/OMS Annotation Information Not Retrieved | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-05-11 06:31:49,532 : XmlGeneration_3 : Initiating XML Generation | H | CAP |  el | 3 | emea-combined-h-2494-el_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json




Heading Not Found 
 ['q Το φάρμακο αυτό τελεί υπό συμπληρωματική παρακολούθηση. Αυτό θα επιτρέψει το γρήγορο προσδιορισμό νέων πληροφοριών ασφάλειας. Μπορείτε να βοηθήσετε μέσω της αναφοράς πιθανών ανεπιθύμητων ενεργειών που ενδεχομένως παρουσιάζετε. Βλ. τέλος της παραγράφου 4 για τον τρόπο αναφοράς ανεπιθύμητων ενεργειών.', 'Τι είναι το Χ και ποια είναι η χρήση του', 'Μην <πάρετε> <χρησιμοποιήσετε> το Χ', 'Άλλα φάρμακα και Χ', 'Το Χ περιέχει {όνομα(τα) εκδόχου(ων)}', 'Πώς να <πάρετε> <χρησιμοποιήσετε> το Χ', 'Πώς να φυλάσσετε το  Χ', 'Τι περιέχει το Χ', 'Εμφάνιση του Χ και περιεχόμενα της συσκευασίας', 'Κάτοχος Άδειας Κυκλοφορίας και Παρασκευαστής', 'Για οποιαδήποτε πληροφορία σχετικά με το παρόν φαρμακευτικό προϊόν, παρακαλείστε να απευθυνθείτε στον τοπικό αντιπρόσωπο του Κατόχου της Άδειας Κυκλοφορίας:', 'Οι πληροφορίες που ακολουθούν απευθύνονται μόνο σε επαγγελματίες υγείας:']


dict_keys([])
Completed Heading Extraction For File
Starting Document Annotation For File :- emea-com

In [ ]:
convertCollectionToDataFrame(b)

## English

In [ ]:
languageCode = 'en'

### Kalydeco



### SmPC

In [ ]:
documentNumber = 0
docFilter = "SmPC.json"
stopWordFilterLen = 6
stopWordlanguage = 'english'
start=10
end=

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

### Annex II

In [ ]:
documentNumber = 1
docFilter = "ANNEX II.json"
stopWordFilterLen = 6
stopWordlanguage = 'english'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

### Labeling

In [ ]:
documentNumber = 2
docFilter = "ANNEX III.json"
stopWordFilterLen = 6
stopWordlanguage = 'english'
start=0
end=1

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end
              )

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Package Leaflet

In [ ]:
documentNumber = 3
docFilter = "LEAFLET.json"
stopWordFilterLen = 100
stopWordlanguage = 'english'
start=0
end=1
isPackageLeaflet = True

In [ ]:
a,b,c = parseDocuments(procedureType,
               languageCode,
               documentNumber,
               docFilter,
               fileNameQrd,
               fileNameMatchRuleBook,
               fileNameDocumentTypeNames,
               stopWordFilterLen,
               start,
               end,
               isPackageLeaflet,
               "Kalydeco"
              )

In [ ]:
convertCollectionToDataFrame(b)

In [ ]:
logger = MatchLogger("MatchLogger",'Kalydeco II-86-PI-clean_ PACKAGE LEAFLET.json', 'CAP', 'en', 'SmPC', fileNameLog = os.path.join(os.path.abspath(os.path.join('..')), 'code','utils','matchLog.txt'))

In [ ]:
from match.rulebook.matchRulebook import MatchRuleBook

rules= MatchRuleBook(
            fileNameRuleBook= fileNameMatchRuleBook,
            procedureType= procedureType,
            languageCode= languageCode,
            documentNumber= 2).ruleDict



In [ ]:
from match.matchStrings.matchStrings import MatchStrings

In [ ]:
matchString = MatchStrings(logger, 2, rules, 6, 'german')


In [ ]:
o,t = 'ANGABEN AUF DER ZWISCHENVERPACKUNG','ANGABEN <AUF DER ÄUSSEREN UMHÜLLUNG> <UND> <AUF DEM BEHÄLTNIS> <UND> <AUF DER ZWISCHENVERPACKUNG>'

#o = o.encode()
#print(o.decode())
matchString.matchStrings(o,t,2)

In [ ]:
fileNameDoc